In [1]:
import pandas as pd
import numpy as np
import time
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    PeftModel
)
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

## Выбор модели

Критерии:
* помещается на 2xT4 (использовались мощности kaggle)
* дообучена под русский
* в топе по числу скачиваний (хорошего лидерборда для таких маленьких моделей я не нашел)

In [ ]:
model_name = "Vikhrmodels/QVikhr-3-4B-Instruction"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

input_text = "Напиши краткое описание книги Гарри Поттер."

messages = [
    {"role": "user", "content": input_text},
]

# Tokenize and generate text
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, enable_thinking=False, return_tensors="pt")
output = model.generate(
    input_ids,
    max_length=100,
    do_sample=False
)

# Decode and print result
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


user
Напиши краткое описание книги Гарри Поттер.
assistant
<think>

</think>

**Краткое описание книги "Гарри Поттер и Философский камень" Дж. К. Роулинг:**

В 1991 году магический мальчик Харри Поттер, родившийся в доме, где его не любили, живёт с д


In [ ]:
with open("/kaggle/input/avito-contest-data/dataset_1937770_3.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

data = []
for line in lines[1:]:
    line = line.strip()
    if line:
        parts = line.split(',', 1)
        if len(parts) == 2:
            data.append(parts)
        else:
          print(parts)

task_data = pd.DataFrame(data, columns=['id', 'text_no_spaces'])
task_data.head(10)

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
5,5,работавМосквеудаленно
6,6,куплютелевизорPhilips
7,7,ищугрузчиковдляпереезда
8,8,ремонтквартирподключ
9,9,куплюноутбукHP


In [ ]:
task_data

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
...,...,...
1000,1000,Янеусну.
1001,1001,Весна-яуженегреюпио.
1002,1002,Весна-скоровырастеттрава.
1003,1003,"Весна-выпосмотрите,каккрасиво."


In [ ]:
def restore_spaces(text_without_spaces):
    prompt = f"""Insert spaces in the text:
===================================
{text_without_spaces}
===================================

Strictly follow the rules:
- It is forbidden to remove letters or punctuation marks
- It is forbidden to change words
- It is forbidden to replace letters
- It is forbidden to add new letters or punctuation marks

The only thing you can do is insert spaces where necessary

Write only the text with inserted spaces and nothing more!
    """

    messages = [
        {"role": "system", "content": "Ты - текстовый редактор. Твоя задача - восстанавливать пробелы в тексте там, где они должны быть"},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=50,
        temperature=0.7,
        do_sample=True, # не детерминированно, потому что модель иногда отказывается исполнять просьбу
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    response = response.strip()

    if ":" in response:
        response = response.split(":", 1)[1].strip()

    return response

def find_space_positions(original_text, restored_text):
    space_positions = []
    space_count = 0

    for i, char in enumerate(restored_text):
        if char == ' ':
            pos_in_original = i - space_count
            space_positions.append(pos_in_original)
            space_count += 1

    return space_positions


example = "ищудомвПодмосковье"
restored_text = restore_spaces(example)
for _ in range(19): # даем еще попыток модели
  # модель иногда отказывается выполнять просьбу или пишет ерунду, это эвристически можно проверить
  if len(restored_text) - len(example) >= 10 or len(restored_text) - len(example) <= 0 or example.replace(' ', '').lower() != restored_text.replace(' ', '').lower():
    restored_text = restore_spaces(example)
  else:
    break

print(find_space_positions(example, restored_text))
print(restored_text)

[3, 6, 7]
ищу дом в Подмосковье


In [ ]:
from tqdm import tqdm
import os

results = []
backup_interval = 50
backup_filename = 'restored_text_results_backup_v2.csv'

for index, row in tqdm(task_data.iterrows(), total=len(task_data)):
    text_without_spaces = row['text_no_spaces']

    restored_text = restore_spaces(text_without_spaces)
    for _ in range(19): # даем еще попыток модели
      # модель иногда отказывается выполнять просьбу или пишет ерунду, это эвристически можно проверить
      if len(restored_text) - len(text_without_spaces) >= 12 or len(restored_text) - len(text_without_spaces) <= 0 or text_without_spaces.replace(' ', '').lower() != restored_text.replace(' ', '').lower():
        restored_text = restore_spaces(text_without_spaces)
      else:
        break
    space_positions = find_space_positions(text_without_spaces, restored_text)

    results.append({
        'id': row['id'],
        'original_text': text_without_spaces,
        'restored_text': restored_text,
        'predicted_positions': space_positions
    })

    if (index + 1) % backup_interval == 0:
        temp_df = pd.DataFrame(results)

        if os.path.exists(backup_filename):
            existing_df = pd.read_csv(backup_filename)
            combined_df = pd.concat([existing_df, temp_df], ignore_index=True)
            combined_df.to_csv(backup_filename, index=False)
        else:
            temp_df.to_csv(backup_filename, index=False)

        print(f"Чекпоинт после {index + 1} итераций")

final_df = pd.DataFrame(results)
final_df.to_csv('restored_text_results_v2.csv', index=False)
final_df

In [3]:
submission = final_df[['id', 'predicted_positions']]
submission['predicted_positions'] = submission['predicted_positions'].astype(str)
submission

,id,predicted_positions
0,0,"[10, 12]"
1,1,"[3, 6, 7]"
2,2,"[4, 12, 13, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"
...,...,...
1000,1000,"[1, 3]"
1001,1001,"[6, 16]"
1002,1002,[19]
1003,1003,"[6, 8, 19, 22]"


In [2]:
submission.to_csv('task_data.csv', index = False)
# проверка
submission = pd.read_csv('task_data.csv')
submission

,id,predicted_positions
0,0,"[10, 12]"
1,1,"[3, 6, 7]"
2,2,"[4, 12, 13, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"
...,...,...
1000,1000,"[1, 3]"
1001,1001,"[6, 16]"
1002,1002,[19]
1003,1003,"[6, 8, 19, 22]"
